In [41]:
import pandas as pd
from nltk.tokenize import sent_tokenize, word_tokenize, PunktSentenceTokenizer
from nltk.corpus import stopwords
#nltk.download()
import string
from nltk.stem import PorterStemmer
from nltk import pos_tag, ne_chunk
import nltk
import math

In [508]:
#file with data
f = pd.read_csv(r'C:\Users\mccol\Desktop\Sapienza\ADM\HW3\Airbnb_Texas_Rentals.csv')

In [6]:
#creating the .tsv files
for i in range(f.index.max()+1):
    op = open(r'C:\Users\mccol\Desktop\Sapienza\ADM\HW3\doc\doc_' + str(i) + '.tsv', 'w', encoding="utf-8")
    for j in range(10):
        op.write('%s\t' %f.iloc[i, j])
    op.close()

### CREATING THE VOCABULARY

In [4]:
#FUNCTIONS
def preprocess(l):
    final = []
    for i in l:
        if not((ps.stem(i) in stopWords) or (ps.stem(i) in (string.punctuation) )):
            final.append(ps.stem(i))
    return (final)

def vocabularization(vocabulary, final, index):
    for word in final:
        if not(word in vocabulary):
            vocabulary[word] = index
            index = index + 1
    return(vocabulary, index)

#not used at the moment
def chunking(sentence):
    names = []
    l = []
    for chunk in ne_chunk(pos_tag(word_tokenize(sentence))) : 
        if type(chunk) is nltk.tree.Tree:
            s = ''
            for  i in chunk:
                s = s + ' ' + i[0]
            names.append(s[1:])
        else:
            l.append(chunk[0])
    return(names, l)

In [16]:
stopWords = set(stopwords.words('english'))
ps = PorterStemmer()
string.punctuation = string.punctuation + '–“”’'

vocabulary= {}
index = 0

for i in range(18259):
    
    op = open(r'C:\Users\mccol\Desktop\Sapienza\ADM\HW3\doc\doc_' + str(i) + '.tsv', 'r', encoding="utf-8")
    for line in op:
        ou = line.strip().split('\t')
        sentence = ou[5].replace('\\n', ' ').replace('/', ' ').replace('*', ' ').replace('\\r', ' ').replace('\\t', ' ') + ' ' + ou[8]
    op.close()
        
    #preprocessing data deleting stop words, punctuations, ecc.  
    final = preprocess(word_tokenize(sentence))
    
    # IF  word not in vocabulary -> add the word
    vocabulary, index = vocabularization(vocabulary, final, index)
            
op = open(r'C:\Users\mccol\Desktop\Sapienza\ADM\HW3\vocabulary.txt', 'w', encoding="utf-8")
op.write(str(vocabulary))
op.close()

### INVERTED INDEX

In [17]:
inverted_index = {}

for file in range(18259):

    op = open(r'C:\Users\mccol\Desktop\Sapienza\ADM\HW3\doc\doc_' + str(file) + '.tsv', 'r', encoding="utf-8")
    for line in op:
        ou = line.strip().split('\t')
        sentence = ou[5].replace('\\n', ' ').replace('/', ' ').replace('*', ' ').replace('\\r', ' ').replace('\\t', ' ') + ' ' + ou[8]
    op.close()
 
    
    #preprocessing data deleting stop words, punctuations, ecc.  
    final = preprocess(word_tokenize(sentence))
    
    
    #CREATING INVERTED INDEX
    for word in final:
        index = vocabulary[word]
        if not (index in inverted_index):
            inverted_index[index] = ['doc_' + str(file)]
        elif not('doc_' + str(file) in inverted_index[index]):
            inverted_index[index] = inverted_index[index] + ['doc_' + str(file)]

op = open(r'C:\Users\mccol\Desktop\Sapienza\ADM\HW3\inverted_index.txt', 'w', encoding="utf-8")
op.write(str(inverted_index))
op.close()

In [505]:
user_query = input()

#names, l = chunking(sentence) 
    
#preprocessing data deleting stop words, punctuations, ecc.  
final = preprocess(word_tokenize(user_query))
final

queen netflix


['queen', 'netflix']

In [506]:
voc = {}
inverted_idx = {}
i=0
for word in final: 
    voc[i]  = vocabulary[word]
    i = i+1
for index in range(i):
    inverted_idx[voc[index]] = inverted_index[voc[index]]

In [509]:
#finding list of docs that contain all the words in the query
docs = []

for i in range(18259):
    doc = 'doc_' + str(i)
    b = True
    for j in voc.values(): 
        b = b and (doc in inverted_idx[j])
    if b:
        docs.append(i)

df = f.filter(items = ['title', 'description', 'city', 'url']).loc[docs]
df.description = list(map(lambda x: x.replace('\\n', ' '), df.description.tolist()))
df.style.hide_index()

title,description,city,url
"Next to Stadiums! Updated, Modern 1 BD apt","Cozy and comfortable modern 1 BD apt. This apt is in the middle of EVERYTHING. Tons of restaurants and tourist attractions with in 1-2 miles. A second floor unit overlooking the pool. Comfortable king size Serta memory foam mattress in the bedroom, and a comfortable couch and love seat in the living room, along with a queen size air mattress. High speed internet and Netflix provided. This apartment is the perfect place if you are coming to town for the theme Parks or anything at ATT stadium.",Arlington,https://www.airbnb.com/rooms/18363018?location=Bedford%2C%20TX
Great room on East Downtown,"Nice Bedroom, 2 beds (1 full, 1 twin, soon to be 1 queen in September 2017) with private bathroom, fridge, desk, and microwave...Also wifi, cable, Netflix, and Hulu. Only two blocks from Metro station and less that 10 min to downtown, Minute Maid and Dynamo Stadium!",Houston,https://www.airbnb.com/rooms/15254951?location=Baytown%2C%20TX
"1BR perfect location near UT, downtown, Cherrywood","Location! - Walk to UT campus! - 10 minute drive downtown! Apartment features - 50 inch HD 4k TV Netflix, surround sound, bluetooth - Fully stocked kitchen - Queen size bed with ultra plush mattress - microfiber sheets, antimicrobial silicone pillows - 2 towel sets + 2 additional pool towels Building features - Secure covered parking - Resort-style pool, gas grills, covered lounge with big screen cable TVs - Game room with a pool table, cable TVs - 24 hour gym",Austin,https://www.airbnb.com/rooms/15604021?location=Austin%2C%20TX
Glam Celebrity Home in UP! 4BR. 10% for Charity!,"Host your holiday get together here!!! Enjoy staying in our luxurious new 4 Bedroom, 4.5 Bath University Park 3700 sf home in the prestigious Park Cities! Previous Celebrity Home now can be yours! All bedrooms have attached bathrooms. 5 HD TV's. Netflix. HULU. HBO. Cable. WIFI. 8 beds (1K, 5 Queen, 2 Twin) + 4 inflatables. Whether a business traveler, bridal party, or family, come stay at a home and AirBnB of professional athletes, socialites, and other celebrities! Close to everything!",Dallas,https://www.airbnb.com/rooms/14668342?location=Arlington%2C%20TX
Great room on East Downtown,"Nice Bedroom, 2 beds (1 full, 1 twin, soon to be 1 queen in September 2017) with private bathroom, fridge, desk, and microwave...Also wifi, cable, Netflix, and Hulu. Only two blocks from Metro station and less that 10 min to downtown, Minute Maid and Dynamo Stadium!",Houston,https://www.airbnb.com/rooms/15254951?location=Atascocita%2C%20TX
Luxury Oceanview Condo,"This luxury ocean view condo is a short walk directly onto Whitecap beach on Padre Island! Enjoy beautiful views from your private balcony, LR and MBR. Very tastefully decorated unit has it all! Flat-screen cable TVs in LR and MBR, Netflix and Blueray DVD. High-speed internet in the unit allows you to work or play online at no additional cost. Feel at home for as long as you like with washer and dryer, and clean, modern full kitchen. The beautifully appointed MBR has pillow-top queen bed, walk-in closet and large bathroom with tub and shower. The LR has a queen sleeper sofa, balcony and fireplace. Amenities onsite include a beautiful pool and hot tub and a well equipped fitness center. Pond, inlet and surf-fishing are available on the property or within easy walk. Restaurants and nightlife abound on Padre Island and in nearby Port Aransas. Padre Island National Park and Mustang Island State Park are also close by; everything you could want for an unforgettable beach vacation!",Corpus Christi,https://www.airbnb.com/rooms/1022134?location=Baffin%20Bay%2C%20TX
"Just outside Austin ATX, Queen room","You will have room with queen bed upstairs in my beautiful house, access to sitting areas, two bathrooms, living room w/ Netflix, cool backyard patio/fire pit, kitchen, dining room, laundry, etc. Perfect for business or tourism travelers. Close to Dell, Samsung, Apple, etc. A

### inverted index with tfIdf

In [264]:
inverted_index_2 = {}

for file in range(18259):

    op = open(r'C:\Users\mccol\Desktop\Sapienza\ADM\HW3\doc\doc_' + str(file) + '.tsv', 'r', encoding="utf-8")
    for line in op:
        ou = line.strip().split('\t')
        sentence = ou[5].replace('\\n', ' ').replace('/', ' ').replace('*', ' ').replace('\\r', ' ').replace('\\t', ' ') + ' ' + ou[8]
    op.close()
 
    
    #preprocessing data deleting stop words, punctuations, ecc.  
    final = preprocess(word_tokenize(sentence))
    
    
    #CREATING INVERTED INDEX
    for word in final:
        index = vocabulary[word]
        
        tf = final.count(word) / len(final)
        idf = math.log( 18258 / len(inverted_index[vocabulary[word]]))
        
        if not (index in inverted_index_2):
            inverted_index_2[index] = [('doc_' + str(file), tf*idf )]
        elif not(('doc_' + str(file), tf*idf)  in inverted_index_2[index]):
            inverted_index_2[index] = inverted_index_2[index] + [('doc_' + str(file), tf*idf)]


op = open(r'C:\Users\mccol\Desktop\Sapienza\ADM\HW3\inverted_index_2.txt', 'w', encoding="utf-8")
op.write(str(inverted_index_2))
op.close()

In [385]:
user_query = input()

#preprocessing data deleting stop words, punctuations, ecc.  
final_query = preprocess(word_tokenize(user_query))
final_query

equidistant tranquil


['equidist', 'tranquil']

In [386]:
voc = {}
inverted_query = {}
i=0
for word in final_query: 
    voc[i]  = vocabulary[word]
    i = i+1
for index in range(i):
    inverted_query[voc[index]] = inverted_index_2[voc[index]]

In [475]:
# NOMINATOR FOR COSINE

n = {}
index = 0
for i in inverted_query:
    for j in inverted_query[i]:
        if index == 0:
            n[j[0]] = [j[1]]
        elif (not (j[0] in n)):
            n[j[0]] = [0]*index + ([j[1]])
        else:   
            n[j[0]] = n[j[0]] + [0]*(index - len(n[j[0]])) + ([j[1]])
    index = index + 1

for i in n:
    if len(n[i]) < len(final_query) :
        n[i] = n[i] + [0]*(l-len(n[i]))
n

{'doc_1329': [0.16453015058394022, 0.09828514400499608],
 'doc_1762': [0.12472446899105145, 0],
 'doc_5957': [0.11716541026432106, 0],
 'doc_7560': [0.12472446899105145, 0],
 'doc_9280': [0.11716541026432106, 0],
 'doc_9438': [0.26665231301535136, 0],
 'doc_12492': [0.11716541026432106, 0],
 'doc_15015': [0.26665231301535136, 0],
 'doc_115': [0, 0.21485589619696818],
 'doc_125': [0, 0.40168711028128834],
 'doc_649': [0, 0.12484869643877881],
 'doc_751': [0, 0.17766929877826218],
 'doc_864': [0, 0.12484869643877881],
 'doc_902': [0, 0.08715852392895879],
 'doc_1011': [0, 0.14213543902260972],
 'doc_1083': [0, 0.20084355514064417],
 'doc_1218': [0, 0.08715852392895879],
 'doc_1541': [0, 0.09238803536469632],
 'doc_1606': [0, 0.3079601178823211],
 'doc_1929': [0, 0.08104213628482133],
 'doc_1932': [0, 0.10998575638654323],
 'doc_2046': [0, 0.25663343156860086],
 'doc_2047': [0, 0.14901296026563923],
 'doc_2097': [0, 0.07332383759102883],
 'doc_2157': [0, 0.10042177757032208],
 'doc_2172':

In [389]:
# tfidf of the query
tfidf_query = []  
for word in final_query:
    tf_query = final_query.count(word) / len(final_query)
    idf_query = math.log( 18258 / len(inverted_index[vocabulary[word]]))
    tfidf_query.append(tf_query * idf_query)
tfidf_query

[3.866458538722595, 2.309700884117408]

In [412]:
import numpy
#norm of the query 
norm_query = numpy.sqrt(sum(list(map(lambda x: x**2, tfidf_query))))

In [396]:
#norm of all the docs
norm_doc = {}
for i in range(18259):
    doc = 'doc_'+ str(i)
    nomin = 0
    for i in inverted_query:
        for j in inverted_query[i]:
            if j[0] == doc :
                nomin = nomin + j[1]**2
                norm_doc[doc] = nomin
for i in norm_doc:
    norm_doc[i] = numpy.sqrt(norm_doc[i])
norm_doc

{'doc_115': 0.21485589619696818,
 'doc_125': 0.40168711028128834,
 'doc_649': 0.12484869643877881,
 'doc_751': 0.17766929877826218,
 'doc_864': 0.12484869643877881,
 'doc_902': 0.08715852392895879,
 'doc_1011': 0.14213543902260972,
 'doc_1083': 0.20084355514064417,
 'doc_1218': 0.08715852392895879,
 'doc_1329': 0.19165108917837348,
 'doc_1541': 0.09238803536469632,
 'doc_1606': 0.3079601178823211,
 'doc_1762': 0.12472446899105145,
 'doc_1929': 0.08104213628482133,
 'doc_1932': 0.10998575638654323,
 'doc_2046': 0.25663343156860086,
 'doc_2047': 0.14901296026563923,
 'doc_2097': 0.07332383759102883,
 'doc_2157': 0.10042177757032208,
 'doc_2172': 0.11266833581060527,
 'doc_2215': 0.15398005894116054,
 'doc_2416': 0.8398912305881484,
 'doc_2562': 0.12484869643877881,
 'doc_2670': 0.06327947627718926,
 'doc_2822': 0.0905765052595062,
 'doc_2827': 0.14901296026563923,
 'doc_3047': 0.12484869643877881,
 'doc_3063': 0.10265337262744036,
 'doc_3083': 0.07699002947058027,
 'doc_3091': 0.06599145

In [478]:
cos = numpy.dot(tfidf_query, n['doc_1762'])
cosine = (cos / (norm_doc['doc_1762']*norm_query))
cosine

0.8584879495822157

In [476]:
from scipy.spatial import distance
cosine = 1 - distance.cosine(tfidf_query, n['doc_1762'])
cosine

0.8584879495822157

In [500]:
daje= []
for i in n:
    st = i
    cos = numpy.dot(tfidf_query, n[st])
    cosine = round(cos / (norm_doc[st]*norm_query), 10)
    daje.append((cosine, i ))
    print('cosine for ' + i + ' whit words ['+ user_query + '] is:', cosine)

cosine for doc_1329 whit words [equidistant tranquil] is: 1.0
cosine for doc_1762 whit words [equidistant tranquil] is: 0.8584879496
cosine for doc_5957 whit words [equidistant tranquil] is: 0.8584879496
cosine for doc_7560 whit words [equidistant tranquil] is: 0.8584879496
cosine for doc_9280 whit words [equidistant tranquil] is: 0.8584879496
cosine for doc_9438 whit words [equidistant tranquil] is: 0.8584879496
cosine for doc_12492 whit words [equidistant tranquil] is: 0.8584879496
cosine for doc_15015 whit words [equidistant tranquil] is: 0.8584879496
cosine for doc_115 whit words [equidistant tranquil] is: 0.5128337357
cosine for doc_125 whit words [equidistant tranquil] is: 0.5128337357
cosine for doc_649 whit words [equidistant tranquil] is: 0.5128337357
cosine for doc_751 whit words [equidistant tranquil] is: 0.5128337357
cosine for doc_864 whit words [equidistant tranquil] is: 0.5128337357
cosine for doc_902 whit words [equidistant tranquil] is: 0.5128337357
cosine for doc_1011

In [504]:
import heapq

heap = []
for i in daje:
    heapq.heappush(heap, i)

heapq.nlargest(5, heap)

[(1.0, 'doc_1329'),
 (0.8584879496, 'doc_9438'),
 (0.8584879496, 'doc_9280'),
 (0.8584879496, 'doc_7560'),
 (0.8584879496, 'doc_5957')]